# Oil Spill Segmentation - Model Development (Minimal)
## U-Net Implementation for Binary Segmentation

This notebook implements a minimal U-Net model for oil spill segmentation.

In [1]:
# Essential imports only
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import gc
import os
from pathlib import Path

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("CUDA memory:", torch.cuda.get_device_properties(0).total_memory // 1024**3, "GB")

PyTorch version: 2.8.0+cpu
CUDA available: False


In [2]:
# Load configuration from dataset pipeline
try:
    # Load from correct config folder and file name
    with open('config/dataset_config.json', 'r') as f:
        config = json.load(f)
    print("Loaded configuration from dataset pipeline")
    print(f"Image size: {config.get('image_size', 'Not found')}")
    print(f"Batch size: {config.get('batch_size', 'Not found')}")
except Exception as e:
    print(f"Could not load config: {e}")
    # Removed dummy config fallback, require real config
    raise Exception("Dataset config required! Run dataset pipeline first.")

# Use GPU if available, CPU as fallback
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Clear any existing GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

Loaded configuration from dataset pipeline
Image size: 256
Batch size: 2
Using device: cpu


26

## U-Net Architecture Implementation
Minimal U-Net with encoder-decoder structure for binary segmentation.

In [3]:
class DoubleConv(nn.Module):
    """Double convolution block used in U-Net"""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
    
    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # Handle size mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

print("U-Net building blocks defined successfully")

U-Net building blocks defined successfully


In [4]:
class UNet(nn.Module):
    """Minimal U-Net for binary segmentation"""
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        
        # Encoder (downsampling path)
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        
        # Decoder (upsampling path)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        
        # Output layer
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)
        
    def forward(self, x):
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        # Decoder with skip connections
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        
        # Output
        logits = self.outc(x)
        return logits

print("U-Net model class defined successfully")

U-Net model class defined successfully


## Model Initialization and Testing

In [5]:
# Initialize model
print("Initializing U-Net model...")
model = UNet(n_channels=3, n_classes=1)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {total_params * 4 / 1024**2:.2f} MB")

# Move to device
model = model.to(device)
print(f"Model moved to {device}")

# Memory cleanup
gc.collect()

Initializing U-Net model...
Total parameters: 31,037,633
Trainable parameters: 31,037,633
Model size: 118.40 MB
Model moved to cpu


0

In [ ]:
# Test model with sample input
print("Testing model with sample input...")

# Use config values for testing
batch_size = 1  # Small batch for testing
image_size = config.get('image_size', 256)

test_input = torch.randn(batch_size, 3, image_size, image_size).to(device)
print(f"Test input shape: {test_input.shape}")

# Forward pass
model.eval()
with torch.no_grad():
    output = model(test_input)

print(f"Model output shape: {output.shape}")
print(f"Output range: [{output.min().item():.4f}, {output.max().item():.4f}]")

# Apply sigmoid for binary segmentation
prob_output = torch.sigmoid(output)
print(f"Probability output range: [{prob_output.min().item():.4f}, {prob_output.max().item():.4f}]")

print("✓ Model test successful!")

# Cleanup
del test_input, output, prob_output
gc.collect()

Testing model with sample input...
Test input shape: torch.Size([1, 3, 256, 256])
Model output shape: torch.Size([1, 1, 256, 256])
Output range: [-0.1246, -0.0601]
Probability output range: [0.4689, 0.4850]
✓ Model test successful!


0

## Loss Functions for Binary Segmentation
Implementing Binary Cross Entropy + Dice Loss.

In [7]:
class DiceLoss(nn.Module):
    """Dice Loss for binary segmentation"""
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        # Apply sigmoid to predictions
        predictions = torch.sigmoid(predictions)
        
        # Flatten tensors
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        
        # Calculate Dice coefficient
        intersection = (predictions * targets).sum()
        dice = (2. * intersection + self.smooth) / (predictions.sum() + targets.sum() + self.smooth)
        
        return 1 - dice

class CombinedLoss(nn.Module):
    """Binary Cross Entropy + Dice Loss"""
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.dice_loss = DiceLoss()
    
    def forward(self, predictions, targets):
        bce = self.bce_loss(predictions, targets)
        dice = self.dice_loss(predictions, targets)
        return self.bce_weight * bce + self.dice_weight * dice

print("Loss functions defined successfully")

Loss functions defined successfully


In [8]:
# Test loss functions with real dimensions
print("Testing loss functions...")

# Use actual config dimensions for testing
batch_size = 1
image_size = config.get('image_size', 256)

# Use real config dimensions instead of dummy small sizes
test_pred = torch.randn(batch_size, 1, image_size, image_size).to(device)
test_target = torch.randint(0, 2, (batch_size, 1, image_size, image_size)).float().to(device)

print(f"Prediction shape: {test_pred.shape}")
print(f"Target shape: {test_target.shape}")

# Test individual losses
bce_loss = nn.BCEWithLogitsLoss()
dice_loss = DiceLoss()
combined_loss = CombinedLoss()

bce_val = bce_loss(test_pred, test_target)
dice_val = dice_loss(test_pred, test_target)
combined_val = combined_loss(test_pred, test_target)

print(f"BCE Loss: {bce_val.item():.4f}")
print(f"Dice Loss: {dice_val.item():.4f}")
print(f"Combined Loss: {combined_val.item():.4f}")

print("✓ Loss functions test successful!")

# Cleanup
del test_pred, test_target, bce_val, dice_val, combined_val
gc.collect()

Testing loss functions...
Prediction shape: torch.Size([1, 1, 256, 256])
Target shape: torch.Size([1, 1, 256, 256])
BCE Loss: 0.8043
Dice Loss: 0.5007
Combined Loss: 0.6525
✓ Loss functions test successful!


0

## Model Configuration Export
Save model configuration for training notebook.

In [ ]:
# Update configuration with model settings
model_config = {
    'model_name': 'UNet',
    'n_channels': 3,
    'n_classes': 1,
    'total_parameters': total_params,
    'trainable_parameters': trainable_params,
    'loss_function': 'BCE_Dice_Combined',
    'bce_weight': 0.5,
    'dice_weight': 0.5,
    'device': str(device)
}

# Merge with existing config
config.update(model_config)

# Save to config folder with proper naming
os.makedirs('config', exist_ok=True)
with open('config/model_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("Configuration updated and saved successfully")
print("Ready for training notebook!")

# Final cleanup and summary
gc.collect()
print("MODEL DEVELOPMENT COMPLETE")
print("✓ U-Net architecture implemented")
print("✓ Binary Cross Entropy + Dice Loss ready")
print("✓ Model tested successfully")
print("✓ Configuration saved for training")
print("✓ Ready for real data training")
print("\nNext: Run training & evaluation notebook")

Configuration updated and saved successfully
Ready for training notebook!

MODEL DEVELOPMENT COMPLETE
✓ U-Net architecture implemented
✓ Binary Cross Entropy + Dice Loss ready
✓ Model tested successfully
✓ Configuration saved for training
✓ Ready for real data training

Next: Run training & evaluation notebook
